# D4
## ISTAT SDMX - Net income on 1st January

[See on IstatData]
// NOT PRESENT(https://esploradati.istat.it/databrowser/#/it/dw/categories/IT1,POP,1.0/POP_FOREIGNIM/DCIS_POPSTRRES1/IT1,29_7_DF_DCIS_POPSTRRES1_1,1.0)

In [1]:
#!pip install pandasdmx requests requests_cache xmltodict

In [2]:
import pandas as pd
import pandasdmx as sdmx
import json
import requests
# from pandasdmx import Request
import xmltodict
from datetime import datetime
import os

In [3]:
# 1 -  EXPLORE DATASTRUCTURE
response = requests.get('http://sdmx.istat.it/SDMXWS/rest/datastructure/IT1/DCCV_REDNETFAMFONTERED/')
print(response.status_code)

if response.status_code == 200:
    content = response.content
    
    if len(content) > 0:
        try:
            xml_data = xmltodict.parse(content)
            json_string_data = json.dumps(xml_data,
                                    allow_nan = True, # If we hadn't set allow_nan to
                                                      # true we would have got
                                                      # ValueError: Out of range float
                                                      # values are not JSON compliant
                                    indent = 6) # Indentation can be used for pretty-printing
            # Now you can work with the parsed JSON data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Empty content received.")
else:
    print("Request failed with status code:", response.status_code)

print(json_string_data)
type(json_string_data)

200
{
      "message:Structure": {
            "@xmlns:message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
            "@xmlns:structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
            "@xmlns:common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
            "message:Header": {
                  "message:ID": "IDREF37",
                  "message:Test": "false",
                  "message:Prepared": "2023-10-14T17:14:53.4483124+02:00",
                  "message:Sender": {
                        "@id": "Unknown"
                  },
                  "message:Receiver": {
                        "@id": "Unknown"
                  }
            },
            "message:Structures": {
                  "structure:DataStructures": {
                        "structure:DataStructure": {
                              "@id": "DCCV_REDNETFAMFONTERED",
                              "@urn": "urn:sdmx:org.sdmx.infomodel.datastructure.

str

By querying the API, we will obtain an XML output that includes the structure:DimensionList tag, which contains the list of dimensions, i.e., the data schema of the dataset. In our case, the dimensions are as follows: `FREQ`, `ITTER107`, `COND_PROF_PREV_PPERC`, `SESSO`, `NUM_COMPONENTI`, `PRES_AFF_IMP`, `FONTE_REDPRINC`, `SESSO_PPERC`, `TIPOLOGIA_FAM`, `FAM_ANZIANI`, `FAM_MINORI`,  `TIPO_DATO`, `TITOLO_STUDIO_PPERC`.

To understand the meaning of this abbreviations we can look at the package called `codelist`. It can be queried by the previous discovered IDs. Let's for example explore `TIPOLOGIA_FAM`. Reading the XML above we see that the ID to query relative to the package `codelist` is: `CL_TIPOLOGIA_FAMILIARE1`. Querying the URL `http://sdmx.istat.it/SDMXWS/rest/codelist/IT1/CL_TIPOLOGIA_FAMILIARE1` in Postman (some API response are too long to be loaded in a Jupyter Notebook) we can see that this dimension is relative to the territory of Italy and all Italian Municipality are listed with full name and ID. An example of one record is shown below:


`<structure:Code id="SLL_2011_116" urn="urn:sdmx:org.sdmx.infomodel.codelist.Code=IT1:CL_ITTER107(5.6).SLL_2011_116">`
<br>
`    <common:Name xml:lang="it">Fossano</common:Name>`
    <br>
`    <common:Name xml:lang="en">Fossano</common:Name>`
    <br>
`    <structure:Parent>`
    <br>
`        <Ref id="IT" />`
        <br>
`    </structure:Parent>`
    <br>
`</structure:Code>`

Since we are interested in all kinds of family, we find that `99` is the ID we will need for our API request for this dimension:Code>

In [7]:
# 2 - Explore the meaning of the dimensions of the dataset

response = requests.get('http://sdmx.istat.it/SDMXWS/rest/codelist/IT1/CL_TIPOLOGIA_FAMILIARE1')
print(response.status_code)

if response.status_code == 200:
    content = response.content
    
    if len(content) > 0:
        try:
            xml_data = xmltodict.parse(content)
            json_string_data = json.dumps(xml_data,
                                    allow_nan = True, # If we hadn't set allow_nan to
                                                      # true we would have got
                                                      # ValueError: Out of range float
                                                      # values are not JSON compliant
                                    indent = 6) # Indentation can be used for pretty-printing
            # Now you can work with the parsed JSON data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Empty content received.")
else:
    print("Request failed with status code:", response.status_code)

print(json_string_data)

200
{
      "message:Structure": {
            "@xmlns:message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
            "@xmlns:structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
            "@xmlns:common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
            "message:Header": {
                  "message:ID": "IDREF40",
                  "message:Test": "false",
                  "message:Prepared": "2023-10-14T17:30:23.0695452+02:00",
                  "message:Sender": {
                        "@id": "Unknown"
                  },
                  "message:Receiver": {
                        "@id": "Unknown"
                  }
            },
            "message:Structures": {
                  "structure:Codelists": {
                        "structure:Codelist": {
                              "@id": "CL_TIPOLOGIA_FAMILIARE1",
                              "@urn": "urn:sdmx:org.sdmx.infomodel.codelist.Codelist=IT1:C

We explore all the other dimensions and since for our purpose we need all the ages we will have to understand how the are divided and which are the values. In order to do that we need to know the ID of our datasets. We can checkit in the URL on IstatData, and we see that is `29_7`.
We can query the API rest service like this to obtain our result:

In [11]:
# 3 -  EXPLORE VALUES IN DIMENSIONS
response = requests.get('http://sdmx.istat.it/SDMXWS/rest/availableconstraint/29_7')
print(response.status_code)

if response.status_code == 200:
    content = response.content
    
    if len(content) > 0:
        try:
            xml_data = xmltodict.parse(content)
            json_string_data = json.dumps(xml_data,
                                    allow_nan = True, # If we hadn't set allow_nan to
                                                      # true we would have got
                                                      # ValueError: Out of range float
                                                      # values are not JSON compliant
                                    indent = 6) # Indentation can be used for pretty-printing
            # Now you can work with the parsed JSON data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Empty content received.")
else:
    print("Request failed with status code:", response.status_code)

# Print is disable in documentation since the response is too long to be shown here. Uncomment to see it.
# print(json_string_data)

200


We can now compose our final query for retrive all values relative of all ages, for all Italy, divided by sex.
Our final URL will be: http://sdmx.istat.it/SDMXWS/rest/data/29_7/..IT.1+2./

The filters we apply after the `data` request are: `IT`that is equal to whole Italian nation, and `1+2`where we specify we want the data divided by males (1) and females (2). When you see just the period `.` it means the filter is not specified, so in our case, we are requesting all ages (from `Y0` to `Y_GE100`) of all available time periods (from  2019 to 2022).

In [8]:
# 4 -  FINAL QUERY WITH FILTERS
response = requests.get('http://sdmx.istat.it/SDMXWS/rest/data/32_292/A.TOTAL.ITI+ITF.9.99.99.2.9.9.99.9.9..99')
print(response.status_code)

if response.status_code == 200:
    content = response.content
    
    if len(content) > 0:
        try:
            xml_data = xmltodict.parse(content)
            json_string_data = json.dumps(xml_data,
                                    allow_nan = True, # If we hadn't set allow_nan to
                                                      # true we would have got
                                                      # ValueError: Out of range float
                                                      # values are not JSON compliant
                                    indent = 6) # Indentation can be used for pretty-printing
            # Now you can work with the parsed JSON data
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)
    else:
        print("Empty content received.")
else:
    print("Request failed with status code:", response.status_code)

# Print is disable in documentation since the response is too long to be shown here. Uncomment to see it.
# print(json_string_data)

200


Now we crate a well formed JSON sting from the response.

The code snippet performs the following tasks:

1. It takes a JSON string called `json_string_data` and creates a nested dictionary, `nested_dict`, using the `json.loads()` function. This step is essential to process and extract information from the JSON data.

2. It defines a translation dictionary, `sex_translation`, which maps numeric codes to corresponding gender labels ('1' to 'Male' and '2' to 'Female'). This dictionary will be used to translate the sex values later.

3. It initializes an empty list, `result`, which will store the extracted information from the nested dictionary.

4. It iterates over the series data in the nested dictionary. Each series represents a set of observations for a specific combination of variables.

5. Within each series, it retrieves the territory and sex values by searching for specific keys ('ITTER107' and 'SESSO') in the series key. If found, it assigns the corresponding values to the `territory` and `sex` variables, respectively. The sex value is translated using the `sex_translation` dictionary.

6. It retrieves the observation values (`obs_values`) for each series and iterates over them. Each observation contains information about the year, age, and quantity.

7. It creates an entry dictionary that contains the extracted information, including the territory, year, sex, age, and quantity.

8. The entry dictionary is appended to the `result` list.

9. Finally, the `result` list is converted to a JSON string, `immigrants_demographic`, using `json.dumps()`. The type of the `immigrants_demographic` variable is printed to verify that it is a string.

In summary, sintethis code processes the nested dictionary, extracts specific information, translates values, and organizes the extracted data into a clean JSON format suitable for visualization or further analysis.

In [33]:
# Creating a nested dictonary from the response in order to create a clean JSON for our visualization
nested_dict = json.loads(json_string_data)

# Translation dictionary
sex_translation = {
    '1': 'Male',
    '2': 'Female'
}

# Extracting information
result = []

series_data = nested_dict['message:GenericData']['message:DataSet']['generic:Series']
for series in series_data:
    series_key = series['generic:SeriesKey']
    territory = None
    sex = None

    for value in series_key['generic:Value']:
        if value['@id'] == 'ITTER107':
            territory = value['@value']
        elif value['@id'] == 'SESSO':
            sex_value = value['@value']
            sex = sex_translation.get(sex_value)

    obs_values = series['generic:Obs']
    for obs in obs_values:
        year = obs['generic:ObsDimension']['@value']
        age = series_key['generic:Value'][1]['@value']
        quantity = obs['generic:ObsValue']['@value']

        entry = {
            'Territory': territory,
            'Year': int(year),
            'Sex': sex,
            'Age': age,
            'Quantity': int(quantity)
        }
        result.append(entry)

# Convert result to JSON
immigrants_demographic = json.dumps(result)
print(type(immigrants_demographic))

<class 'str'>


The next code snippet performs the following tasks:

1. It converts the `immigrants_demographic` string, which contains a JSON representation, into a JSON object using `json.loads()`. This step allows easier manipulation and access to the data.

2. It specifies the folder path where the resulting JSON file will be saved. In this case, the folder path is "../_datasets/Clean".

3. It creates the specified folder if it does not already exist using `os.makedirs()`. This ensures that the folder is available to store the JSON file.

4. It defines the filename for the JSON file as "immigrants_demographic.json".

5. It generates the complete file path by joining the folder path and filename using `os.path.join()`.

6. It saves the `immigrants_demographic_json` JSON object to a file specified by the file path. This is achieved using `json.dump()` with the file opened in write mode ("w").

7. The JSON data is formatted with an indent of 4 spaces to improve readability within the file.

8. Finally, it prints a message confirming the successful saving of the JSON data, along with the file path where it was saved.

So basically this code snippet takes a JSON object, saves it as a JSON file in a specified directory, and provides feedback on the successful saving of the file.

In [9]:

# Convert immigrants_demographic string to JSON object
immigrants_demographic_json = json.loads(immigrants_demographic)

# Specify the folder path to save the JSON file
folder_path = "../_datasets/Clean"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the filename for the JSON file
filename = "net_income.json"

# Generate the file path
file_path = os.path.join(folder_path, filename)

# Save the immigrants_demographic JSON object to file
with open(file_path, "w") as file:
    json.dump(immigrants_demographic_json, file, indent=4)

print(f"JSON data saved to: {file_path}")

NameError: name 'immigrants_demographic' is not defined

The last code snippet loads a JSON file, modifies the "Age" values by removing a "Y" prefix if present, and saves the modified data back to the same JSON file.

In [35]:
import json
import os

# Specify the path to the JSON file
file_path = "../_datasets/Clean/immigrants_demographic.json"

# Load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Iterate over the entries and modify the Age values
for entry in data:
    age = entry["Age"]
    if age.startswith("Y") and age[1:].isdigit():
        entry["Age"] = age[1:]

# Save the modified data back to the JSON file
with open(file_path, "w") as file:
    json.dump(data, file, indent=4)

print(f"JSON data saved to: {file_path}")


JSON data saved to: ../_datasets/Clean/immigrants_demographic.json
